Короткий опис роботи з SQL Alchemy

In [10]:
from sqlalchemy import Column
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship

Base = declarative_base()

class User(Base):
    __tablename__ = "user_account"
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship(
        "Address", back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)
    user = relationship("User", back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

from sqlalchemy import create_engine
engine = create_engine("sqlite://", echo=True, future=True)

In [12]:
Base.metadata.create_all(engine)


In [14]:
from sqlalchemy.orm import Session

# Потім ми передаємо їх до бази даних за допомогою об’єкта під 
# назвою Session , який використовує Engineдля взаємодії з базою
# даних. Метод Session.add_all()використовується тут для додавання
# кількох об’єктів одночасно, і Session.commit()метод буде використано
# для очищення будь-яких очікуваних змін у базі даних, а потім фіксації 
# поточної транзакції бази даних, яка завжди виконується, коли Session 
# використовується

with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    session.add_all([spongebob, sandy, patrick])
    session.commit()


In [15]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
    print(user)


User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')
User(id=4, name='spongebob', fullname='Spongebob Squarepants')
User(id=5, name='sandy', fullname='Sandy Cheeks')


In [7]:
stmt = (
 select(Address)
 .join(Address.user)
 .where(User.name == "sandy")
 .where(Address.email_address == "sandy@sqlalchemy.org")
)
sandy_address = session.scalars(stmt).one()


sandy_address

2022-09-15 13:24:28,907 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? AND address.email_address = ?
2022-09-15 13:24:28,909 INFO sqlalchemy.engine.Engine [cached since 6.533s ago] ('sandy', 'sandy@sqlalchemy.org')


Address(id=2, email_address='sandy@sqlalchemy.org')

In [8]:
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()


patrick.addresses.append(
    Address(email_address="patrickstar@sqlalchemy.org")
)


sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"

session.commit()


2022-09-15 13:25:56,321 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-09-15 13:25:56,328 INFO sqlalchemy.engine.Engine [generated in 0.00740s] ('patrick',)
2022-09-15 13:25:56,336 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-09-15 13:25:56,338 INFO sqlalchemy.engine.Engine [generated in 0.00234s] (3,)
2022-09-15 13:25:56,349 INFO sqlalchemy.engine.Engine UPDATE address SET email_address=? WHERE address.id = ?
2022-09-15 13:25:56,353 INFO sqlalchemy.engine.Engine [generated in 0.00438s] ('sandy_cheeks@sqlalchemy.org', 2)
2022-09-15 13:25:56,356 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-09-15 13:25:56,362 INFO sqlalchemy.engine.Engine [cached since 291.2s ago] ('patrickstar@sqlalchemy.org', 3

In [ ]:
sandy = session.get(User, 2)


sandy.addresses.remove(sandy_address)


In [ ]:
session.flush()


In [ ]:
session.delete(patrick)


Цілий туторіал по SQL Alchemy 

https://docs.sqlalchemy.org/en/14/tutorial/index.html#unified-tutorial

Початком будь-якої програми SQLAlchemy є об’єкт під назвою Engine. Цей об’єкт діє як центральне джерело з’єднань із певною базою даних, надаючи як фабрику, так і простір зберігання, який називається пулом з’єднань для цих з’єднань з базою даних. Механізм, як правило, є глобальним об’єктом, створеним лише один раз для певного сервера бази даних і налаштованим за допомогою рядка URL-адреси, який описуватиме, як він має з’єднуватися з хостом бази даних або серверною частиною.

Коли Engine, вперше повернутий create_engine(), фактично ще не намагався підключитися до бази даних; це відбувається лише в перший раз, коли його просять виконати завдання щодо бази даних. Це шаблон розробки програмного забезпечення, відомий як відкладена ініціалізація .

Ми також вказали параметр create_engine.echo, який буде вказувати Engineжурналювати весь SQL, який він видає, до реєстратора Python, який записуватиме стандартний вихід. Цей прапорець є скороченим способом більш формального налаштування журналювання Python і корисний для експериментів зі сценаріями. Багато прикладів SQL включатимуть цей вихід журналу SQL під [SQL]посиланням, натиснувши яке, ви побачите повну взаємодію SQL


In [9]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [17]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())




[('hello world',)]


Транзакція не виконується автоматично ; коли ми хочемо зафіксувати дані, нам зазвичай потрібно викликати Connection.commit() , як ми побачимо в наступному розділі.

In [ ]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}]
    )
    conn.commit()


In [22]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
    )


In [23]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")




x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
x: 6  y: 8
x: 9  y: 10


In [25]:
with engine.connect() as conn:
    result = conn.execute(
        text("SELECT x, y FROM some_table WHERE y > :y"),
        {"y": 2}
    )
    for row in result:
       print(f"x: {row.x}  y: {row.y}")




x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
x: 6  y: 8
x: 9  y: 10


In [27]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}]
    )
    conn.commit()


In [28]:
stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")




x: 6  y: 8
x: 6  y: 8
x: 9  y: 10
x: 9  y: 10
x: 11  y: 12
x: 11  y: 12
x: 13  y: 14
x: 13  y: 14


In [29]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")




x: 6  y: 8
x: 6  y: 8
x: 9  y: 10
x: 9  y: 10
x: 11  y: 12
x: 11  y: 12
x: 13  y: 14
x: 13  y: 14


In [30]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y":11}, {"x": 13, "y": 15}]
    )
    session.commit()


In [35]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [36]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String)
)

Column- представляє стовпець у таблиці бази даних і присвоює себе Tableоб’єкту. Зазвичай Column містить назву рядка та тип об’єкта. Доступ до набору Columnоб’єктів у термінах батьківського елемента Table зазвичай здійснюється через асоціативний масив, розташований за адресою Table.c

In [37]:
user_table.c.name

user_table.c.keys()

['id', 'name', 'fullname']

In [38]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [39]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('user_id', ForeignKey('user_account.id'), nullable=False),
    Column('email_address', String, nullable=False)
)

У наведеній вище таблиці також міститься третій вид обмеження, яке в SQL є обмеженням «NOT NULL», указане вище за допомогою Column.nullable параметра

In [41]:
metadata_obj.create_all(engine)


In [42]:
from sqlalchemy.orm import registry
mapper_registry = registry()

In [43]:
mapper_registry.metadata

MetaData()

In [45]:
Base = mapper_registry.generate_base()

In [46]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()

In [47]:
from sqlalchemy.orm import relationship
class User(Base):
    __tablename__ = 'user_account'
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address", back_populates="user")
    def __repr__(self):
       return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('user_account.id'))
    user = relationship("User", back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

ми надали метод __repr__() — це абсолютно необов’язковий метод , який суто призначений для того, щоб наші користувальницькі класи мали описове представлення рядків і не потрібні іншим чином

In [48]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [49]:
sandy = User(name="sandy", fullname="Sandy Cheeks")

In [50]:
# emit CREATE statements given ORM registry
mapper_registry.metadata.create_all(engine)

# the identical MetaData object is also present on the
# declarative base
Base.metadata.create_all(engine)

In [51]:
mapper_registry = registry()
Base = mapper_registry.generate_base()

class User(Base):
    __table__ = user_table

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User({self.name!r}, {self.fullname!r})"

class Address(Base):
    __table__ = address_table

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address({self.email_address!r})"

In [52]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)


In [53]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

In [54]:
from sqlalchemy import insert
stmt = insert(user_table).values(name='spongebob', fullname="Spongebob Squarepants")

In [55]:
print(stmt)


INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [57]:
compiled = stmt.compile()
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [58]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()


In [59]:
result.inserted_primary_key

(7,)

In [60]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"}
        ]
    )
    conn.commit()


In [61]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt)


INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


Продовжте огляд всього цього детально в документації на сайті, там розглянуто виняткові ситуації під час реалізації.

https://lectureswww.readthedocs.io/6.www.sync/2.codding/9.databases/2.sqlalchemy/